# Cat vs. Dog Classification
### Colin Santos and Cynthia Lai
---

Loading in Train and Test Images
-----

In [1]:
import os
import numpy as np
from tqdm import tqdm 
from PIL import Image

In [2]:
def get_train_label(file_name):
    """Gives binary based on image name"""
    category = file_name.split('.')[0]
    if   category == 'cat': return 0 # [1, 0]
    elif category == 'dog': return 1 # [0, 1]

In [4]:
train_directory = 'train1/'

# Form list of training images names
train_dir_files  = os.listdir(train_directory)

# Remove hidden MAC files
train_dir_files     = [i for i in train_dir_files if i!= '.DS_Store' ]

# Convert to 224 x 224 images
train_im_list = [Image.open(train_directory + im).resize((224,224)) for im in train_dir_files]

In [5]:
numPics = len(train_dir_files)

In [6]:
# Get training labels 
train_labels = [get_train_label(file) for file in train_dir_files]

In [7]:
(train_labels[0:5], train_labels[20000:20005])

([0, 0, 0, 0, 0], [])

In [9]:
# save the image list for later use
# import _pickle as cPickle
import pickle as cPickle
f = open("im_list.pl", 'wb') # .pl is a pickle file
cPickle.dump(train_im_list, f)
f.close()

In [ ]:
# Formatted data as (numpy arrays of) list of tuples of modified image and respective label or number
train_data = [(np.array(train_im, dtype=np.float64), np.array(train_label)) for (train_im, train_label) in zip(train_im_list, train_labels)]

# Convolutional Neural Network

In [8]:
import tensorflow as tf
import keras
from imagenet_utils import decode_predictions
from imagenet_utils import preprocess_input
from vgg16 import VGG16

Using TensorFlow backend.


## Unmodified VGG16 and ResNet50

#### First, work with VGG16 model.

In [9]:
# train_data and test_data are 2 lists of np.array images
# need to preprocess images a bit more first

# make img file (1, 224, 224, 3)
temp = [np.expand_dims(image, axis = 0) for image in train_dat]
train_data2 = [preprocess_input(image, dim_ordering =  'tf') for image in temp]
#test_data = [preprocess_input(np.expand_dims(image, axis = 0)) for image in test_data]

In [10]:
# load model
model = VGG16(weights="imagenet")

In [17]:
# classify the image
preds = [model.predict(image) for image in tqdm(train_data2[251:750])]

ValueError: too many values to unpack (expected 2)

In [46]:
top_results = [decode_predictions(pred)[0][0] for pred in preds]
labs = [res[1] for res in top_results]
confs = [conf[2] for conf in top_results]

In [42]:
labs

['Siberian_husky',
 'shield',
 'Arctic_fox',
 'plastic_bag',
 'Egyptian_cat',
 'Egyptian_cat',
 'tabby',
 'hamper',
 'Persian_cat',
 'Egyptian_cat',
 'tabby',
 'tabby',
 'Egyptian_cat',
 'Persian_cat',
 'Egyptian_cat',
 'bath_towel',
 'tabby',
 'Egyptian_cat',
 'tabby',
 'Siamese_cat',
 'schipperke',
 'web_site',
 'tabby',
 'Egyptian_cat',
 'Egyptian_cat',
 'tabby',
 'Egyptian_cat',
 'Siamese_cat',
 'tabby',
 'Egyptian_cat',
 'lynx',
 'tabby',
 'tub',
 'Egyptian_cat',
 'cello',
 'prairie_chicken',
 'Egyptian_cat',
 'Angora',
 'Siberian_husky',
 'hamster',
 'Angora',
 'Siamese_cat',
 'Siamese_cat',
 'Persian_cat',
 'tiger_cat',
 'Siamese_cat',
 'tabby',
 'tiger_cat',
 'hamper',
 'Egyptian_cat',
 'Egyptian_cat',
 'Egyptian_cat',
 'tabby',
 'Egyptian_cat',
 'lynx',
 'Persian_cat',
 'lighter',
 'Egyptian_cat',
 'tabby',
 'Egyptian_cat',
 'tabby',
 'Egyptian_cat',
 'tabby',
 'Persian_cat',
 'Persian_cat',
 'Egyptian_cat',
 'Egyptian_cat',
 'Angora',
 'dishwasher',
 'Egyptian_cat',
 'tabby',

### Turn specific results into binary (i.e. Samoyed -> Dog)

In [ ]:
# Load in Imagenet classications groupings for cats and dogs
with open('cats.txt') as f:
    cats = [line.rstrip() for line in f]
with open('dogs.txt') as f:
    dogs = [line.rstrip() for line in f]

In [ ]:
def cat_or_dog(full_class_string):
    """Returns generalized classification of cat or dog"""
    if full_class_string in cats: return 'cat'
    if full_class_string in dogs: return 'dog'
    
def one_or_zero(full_class_string):
    """Returns generalize classification of cat or dog as bool"""
    if full_class_string in cats: return 0
    if full_class_string in dogs: return 1
    
from operator import eq
def calculate_accuracy(pred_labels, true_labels):
    return sum(map(eq, pred_labels, true_labels)) / float(len(true_labels))

In [ ]:
simple_labels = [one_or_zero(label) for label in labels]

accuracy = calculate_accuracy(simple_labels, train_labels)
error = 1 - accuracy

### Transfer Learning
Change the last layer of the model to output only 2 categories.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [49]:
# Borrowed from: https://github.com/DeepLearningSandbox/DeepLearningSandbox/blob/master/transfer_learning/fine-tune.py

from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


NB_EPOCHS = 1
SAMPLE_EPOCH = 50
BAT_SIZE = 16
NB_CLASSES = 2
IM_WIDTH = 224
IM_HEIGHT = 224
FC_SIZE = 50 # number of outputs
NUM_TEST = 50

def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(inputs = base_model.input, outputs = predictions)
    return model

def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

def res50_train(train_dir, val_dir):
    
    train_datagen =  ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rescale=1. / 255)



    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IM_WIDTH, IM_HEIGHT), batch_size=BAT_SIZE)

    validation_generator = test_datagen.flow_from_directory(val_dir, target_size=(IM_WIDTH, IM_HEIGHT), batch_size=BAT_SIZE)

    base_model = ResNet50(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, NB_CLASSES)
    setup_to_transfer_learn(model, base_model)

    model.fit_generator(train_generator, epochs = NB_EPOCHS, steps_per_epoch = SAMPLE_EPOCH, \
                                     validation_data=validation_generator, validation_steps = NUM_TEST)
    model.save("res50_train.model")

In [ ]:
res50_train("train_small", "test_small")

Found 50 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Epoch 1/1
38/50 [=====================>........] - ETA: 1116s - loss: 0.7743 - acc: 0.5000 - ETA: 977s - loss: 1.6092 - acc: 0.5000  - ETA: 965s - loss: 1.9093 - acc: 0.5000 - ETA: 753s - loss: 1.5905 - acc: 0.5000 - ETA: 736s - loss: 1.3342 - acc: 0.5875 - ETA: 707s - loss: 1.1603 - acc: 0.6458 - ETA: 686s - loss: 1.0353 - acc: 0.6875 - ETA: 597s - loss: 1.2160 -

In [ ]:
from keras.applications  import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


NB_EPOCHS = 1
SAMPLE_EPOCH = 200
BAT_SIZE = 16
NB_CLASSES = 2
IM_WIDTH = 224
IM_HEIGHT = 224
FC_SIZE = 200 # number of outputs
NUM_TEST = 100

def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(inputs = base_model.input, outputs = predictions)
    return model

def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
def vgg16_train(train_dir, val_dir):
    
    train_datagen =  ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rescale=1. / 255)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IM_WIDTH, IM_HEIGHT), batch_size=BAT_SIZE)

    validation_generator = test_datagen.flow_from_directory(val_dir, target_size=(IM_WIDTH, IM_HEIGHT), batch_size=BAT_SIZE)

    base_model = VGG16(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, NB_CLASSES)
    setup_to_transfer_learn(model, base_model)

    model.fit_generator(train_generator, epochs =NB_EPOCHS, steps_per_epoch = SAMPLE_EPOCH, \
                                     validation_data=validation_generator, validation_steps = NUM_TEST)
    model.save("vgg16_train.model")